# MACD 经典策略（2800.HK）

- 数据：`yfinance` 日线（auto-adjust）
- 交易假设：用当日收盘计算信号，持仓用于下一日收益（close-to-close）
- 成本：按仓位变动收取 `transaction_cost_bp`（基点）

该 notebook 用于与你的 ML 策略做基准对比。


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import MacdConfig, macd_crossover_signal

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
TRANSACTION_COST_BP = 2.0
ALLOW_SHORT = False  # 建议先做 long-only（更贴近实际约束）

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()

sig = macd_crossover_signal(close, MacdConfig(allow_short=ALLOW_SHORT))
# 用 t 的信号持有到 t+1：所以 position 用 sig（不 shift）乘 next_return(t)
next_ret = np.log(close.shift(-1) / close)
simple_ret = np.exp(next_ret) - 1.0

bt_df, stats = equity_and_stats_from_positions(
    simple_return=simple_ret.dropna(),
    position=sig.loc[simple_ret.dropna().index],
    transaction_cost_bp=TRANSACTION_COST_BP,
)

print(pd.Series(stats))

plt.figure(figsize=(12,4))
plt.plot(bt_df["strategy_equity"], label="MACD strategy")
plt.plot(bt_df["buyhold_equity"], label="Buy&Hold", linestyle="--")
plt.title(f"MACD Backtest | {TICKER} | cost={TRANSACTION_COST_BP:.0f}bps | short={ALLOW_SHORT}")
plt.legend()
plt.grid(alpha=0.3)
plt.show()
